In [1]:
# Just testing and exploring the datasets for mapping DataDate-GVKEY-IID to company names
# It creates 1943 duplicates because there are some that are duplicate for some reason.......

In [1]:
import datetime
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Use shared data loader
sys.path.append(str(Path.cwd().parent / 'src'))
from data_loader import load_data

print(f"Start time: {datetime.datetime.now()}")
pd.set_option("mode.chained_assignment", None)

Start time: 2025-10-01 12:16:07.409315


/var/folders/bn/cx_ppcpx0ngbpf4h354g9_k80000gn/T/ipykernel_4741/2414979913.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def merge_company_names_etc(raw, mappings):
    mappings['datadate'] = pd.to_datetime(mappings['datadate'])

    raw['date'] = pd.to_datetime(raw['ret_eom'])

    merged = pd.merge(raw, mappings[['gvkey', 'datadate', 'iid', 'tic', 'conm', 'cusip','cik']], 
                    left_on=['gvkey', 'date', 'iid'], 
                    right_on=['gvkey', 'datadate', 'iid'], 
                    how='left')
    
    # checking duplicates for your info
    # Check for duplicates in the merged DataFrame
    original_row_count = raw.shape[0]
    merged_row_count = merged.shape[0]

    if original_row_count == merged_row_count:
        print("No duplicates were created during the merge.")
    else:
        print(f"Duplicates detected: Original rows = {original_row_count}, Merged rows = {merged_row_count}, Difference = {merged_row_count - original_row_count}")

    return merged

In [3]:
raw = load_data(filename="usa_can_filtered_data.csv", parse_dates=["ret_eom"], low_memory=False)
raw['date'] = pd.to_datetime(raw['ret_eom'])
# raw['iid'] = raw['iid'].astype(str).str.strip().str.upper()

Loading data from: /Users/kevin/Coding Projects/Asset-Management-Hackathon-2025/data/usa_can_filtered_data.csv


In [65]:
# Load data using the shared data loader
print("Loading data...")

mappings = load_data(filename="Global (ex Canada and US) Company Name Merge by DataDate-GVKEY-IID.csv", low_memory=False)
mappings['datadate'] = pd.to_datetime(mappings['datadate'])
mappings['gvkey'] = mappings['gvkey'].astype(float)
# mappings['iid'] = mappings['iid'].astype(str).str.strip().str.upper()

Loading data...
Loading data from: /Users/kevin/Coding Projects/Asset-Management-Hackathon-2025/data/Global (ex Canada and US) Company Name Merge by DataDate-GVKEY-IID.csv


In [78]:
# Load data using the shared data loader
print("Loading data...")

mappings2 = load_data(filename="cik_gvkey_linktable_USA_only.csv", low_memory=False)
mappings2['datadate'] = pd.to_datetime(mappings['datadate'])
mappings2['gvkey'] = mappings['gvkey'].astype(float)

Loading data...
Loading data from: /Users/kevin/Coding Projects/Asset-Management-Hackathon-2025/data/cik_gvkey_linktable_USA_only.csv


In [4]:
# Load data using the shared data loader
print("Loading data...")

mappings3 = load_data(filename="North America Company Name Merge by DataDate-GVKEY-IID.csv", low_memory=False)
mappings3['datadate'] = pd.to_datetime(mappings3['datadate'])
mappings3['gvkey'] = mappings3['gvkey'].astype(float)

Loading data...
Loading data from: /Users/kevin/Coding Projects/Asset-Management-Hackathon-2025/data/North America Company Name Merge by DataDate-GVKEY-IID.csv


In [5]:
merged1 = merge_company_names_etc(raw, mappings3)

No duplicates were created during the merge.


In [9]:
# Check for missing values in 'gvkey', 'datadate', or 'iid'
missing_values = mappings3[mappings3[['gvkey', 'datadate', 'iid']].isna().any(axis=1)]
if not missing_values.empty:
    print(f"Rows with missing values in 'gvkey', 'datadate', or 'iid': {len(missing_values)}")
    display(missing_values)  # Use display() in a notebook to view the rows

# Check if 'gvkey', 'datadate', and 'iid' are unique
duplicates = mappings3[mappings3.duplicated(subset=['gvkey', 'datadate', 'iid'], keep=False)]
if not duplicates.empty:
    print(f"Duplicate rows based on 'gvkey', 'datadate', and 'iid': {len(duplicates)}")
    display(duplicates)  # Use display() in a notebook to view the rows
else:
    print("No duplicates found based on 'gvkey', 'datadate', and 'iid'.")

No duplicates found based on 'gvkey', 'datadate', and 'iid'.


In [10]:
# Check for duplicates in mappings3
if not is_unique:
    # Get one example of a duplicate group
    example_duplicate_key = duplicate_rows[['gvkey', 'datadate', 'iid']].iloc[0]
    gvkey_example = example_duplicate_key['gvkey']
    datadate_example = example_duplicate_key['datadate']
    iid_example = example_duplicate_key['iid']

    # Filter rows for this specific duplicate group
    example_rows = mappings3[
        (mappings3['gvkey'] == gvkey_example) &
        (mappings3['datadate'] == datadate_example) &
        (mappings3['iid'] == iid_example)
    ]

    print("Example duplicate rows in mappings3:")
    print(example_rows)

NameError: name 'is_unique' is not defined

In [101]:
# Merge mappings onto raw
merged = pd.merge(raw, mappings3[['gvkey', 'datadate', 'iid', 'tic', 'conm', 'cusip','cik']], 
                  left_on=['gvkey', 'date', 'iid'], 
                  right_on=['gvkey', 'datadate', 'iid'], 
                  how='left')

In [102]:
# Check for duplicates in the merged DataFrame
original_row_count = raw.shape[0]
merged_row_count = merged.shape[0]

if original_row_count == merged_row_count:
    print("No duplicates were created during the merge.")
else:
    print(f"Duplicates detected: Original rows = {original_row_count}, Merged rows = {merged_row_count}, Difference = {merged_row_count - original_row_count}")

# Optional: Inspect rows if duplicates are detected
if original_row_count != merged_row_count:
    duplicate_rows = merged[merged.duplicated(subset=['gvkey', 'date', 'iid'], keep=False)]
    print("Duplicate rows:")
    print(duplicate_rows)

Duplicates detected: Original rows = 1398807, Merged rows = 1400750, Difference = 1943
Duplicate rows:
                     id       date    ret_eom     gvkey  iid excntry  \
2147         crsp_19227 2005-02-28 2005-02-28       NaN  NaN     USA   
5830         crsp_88258 2005-02-28 2005-02-28       NaN  NaN     USA   
6131         crsp_89139 2005-02-28 2005-02-28       NaN  NaN     USA   
9008         crsp_19227 2005-03-31 2005-03-31       NaN  NaN     USA   
12674        crsp_88258 2005-03-31 2005-03-31       NaN  NaN     USA   
...                 ...        ...        ...       ...  ...     ...   
1399646  comp_063186_01 2025-06-30 2025-06-30   63186.0   01     USA   
1399800  comp_101276_01 2025-06-30 2025-06-30  101276.0   01     USA   
1399801  comp_101276_01 2025-06-30 2025-06-30  101276.0   01     USA   
1399802  comp_101973_01 2025-06-30 2025-06-30  101973.0   01     USA   
1399803  comp_101973_01 2025-06-30 2025-06-30  101973.0   01     USA   

         stock_ret  year  month 

In [93]:
overlapping_iids = set(raw['iid']).intersection(set(mappings3['iid']))
print("Overlapping iids:", overlapping_iids)

Overlapping iids: {'11', '07', '03', '05', '06', '02', '05C', '90', '04', '01C', '12', '07C', '19', '04C', '02C', '03C', '08', '01', '09', '10', '21'}


In [59]:
mappings['iid'].head()

0    01W
1    01W
2    01W
3    01W
4    01W
Name: iid, dtype: object

In [68]:
print("Unique iids in raw:")
print(raw['iid'].unique())

print("Unique iids in mappings:")
print(mappings2['iid'].unique())

Unique iids in raw:
['01C' '02' '04C' '02C' '01' '03C' '03' '05C' '90' nan '05' '04' '08' '06'
 '09' '10' '07' '07C' '19' '11' '12' '21']
Unique iids in mappings:
['01' '02' '01C' '10' '90' '03' '17' '24' '04C' '02C' '15' '91' '06' '04'
 '90C' '05' '03C' '07' '08' '09' '12' '13' '14' '44' '38' '26' '31' '93'
 '06C' '22' '23' '25' '27' '28' '29' '30' '34' '36' '37' '54' '57' '76'
 '92' '11' '16' '18' '19' '08C' '82' '89' '94' '95' '97' '99' '05C' '96'
 '86' '98' '47' '39' '75' '81' '09C' '40' '88' '41' '62' '35' '20' '71'
 '07C' '39C' '11C' '74' '87' '85' '43' '10C' '32' '84' '21' '12C' '13C'
 '33' '14C' '42' '15C' '16C' '17C' '18C' '19C' '20C' '21C' '22C' '23C'
 '24C' '25C' '26C' '27C' '28C' '64' '80' '75C' '49' '29C' '36C' '32C'
 '41C' '37C' '33C' '69' '67' '31C' '64C' '62C' '63C' '66C' '42C' '30C']


In [63]:
# Group by the specified columns
grouped = raw.groupby(['gvkey', 'date', 'iid'])

# Find groups with duplicates
duplicate_groups = grouped.filter(lambda x: len(x) > 1)

# Check if all rows within each group are identical
identical_groups = duplicate_groups.groupby(['gvkey', 'date', 'iid']).apply(
    lambda group: group.nunique().eq(1).all()
)

# Separate identical and non-identical groups
all_identical = identical_groups[identical_groups].index
not_identical = identical_groups[~identical_groups].index

# Print results
print(f"Number of groups with duplicates: {len(identical_groups)}")
print(f"Number of groups where all duplicates are identical: {len(all_identical)}")
print(f"Number of groups where duplicates are not identical: {len(not_identical)}")

# Optionally, inspect non-identical groups
non_identical_rows = duplicate_groups[
    duplicate_groups.set_index(['gvkey', 'date', 'iid']).index.isin(not_identical)
]
print("Non-identical duplicate rows:")
print(non_identical_rows)

Number of groups with duplicates: 0
Number of groups where all duplicates are identical: 0
Number of groups where duplicates are not identical: 0
Non-identical duplicate rows:
Empty DataFrame
Columns: [id, date, ret_eom, gvkey, iid, excntry, stock_ret, year, month, char_date, char_eom, me, prc, market_equity, div12m_me, chcsho_12m, eqnpo_12m, ret_1_0, ret_3_1, ret_6_1, ret_9_1, ret_12_1, ret_12_7, ret_60_12, seas_1_1an, seas_1_1na, seas_2_5an, seas_2_5na, at_gr1, sale_gr1, capx_gr1, inv_gr1, debt_gr3, sale_gr3, capx_gr3, inv_gr1a, lti_gr1a, sti_gr1a, coa_gr1a, col_gr1a, cowc_gr1a, ncoa_gr1a, ncol_gr1a, nncoa_gr1a, fnl_gr1a, nfna_gr1a, tax_gr1a, be_gr1a, ebit_sale, gp_at, cop_at, ope_be, ni_be, ebit_bev, netis_at, eqnetis_at, dbnetis_at, oaccruals_at, oaccruals_ni, taccruals_at, taccruals_ni, noa_at, opex_at, at_turnover, sale_bev, rd_sale, cash_at, sale_emp_gr1, emp_gr1, ni_inc8q, noa_gr1a, ppeinv_gr1a, lnoa_gr1a, capx_gr2, saleq_gr1, niq_be, niq_at, niq_be_chg1, niq_at_chg1, rd5_at, d

/var/folders/bn/cx_ppcpx0ngbpf4h354g9_k80000gn/T/ipykernel_93529/209578681.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  identical_groups = duplicate_groups.groupby(['gvkey', 'date', 'iid']).apply(


In [7]:
# Check for NaN values in the merged columns
merged_columns = ['tic', 'conm', 'cusip', 'cik']

# Find rows where all merged columns are NaN
nan_rows = merged1[merged1[merged_columns].isna().all(axis=1)]

if nan_rows.empty:
    print("No rows in merged1 have NaN for all the merged columns.")
else:
    print(f"Rows with NaN for all merged columns: {len(nan_rows)}")
    display(nan_rows)  # Use display() in a notebook to inspect the rows

Rows with NaN for all merged columns: 935


,id,date,ret_eom,gvkey,iid,excntry,stock_ret,year,month,char_date,...,age,qmj,qmj_prof,qmj_growth,qmj_safety,datadate,tic,conm,cusip,cik
1370,comp_144498_01C,2005-02-28,2005-02-28,144498.0,01C,CAN,-0.395696,2005,2,20050131,...,42,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2147,crsp_19227,2005-02-28,2005-02-28,NaN,NaN,USA,-0.111594,2005,2,20050131,...,124,NaN,NaN,NaN,-1.020464,NaT,NaN,NaN,NaN,NaN
3145,crsp_71985,2005-02-28,2005-02-28,9919.0,01,USA,-0.050633,2005,2,20050131,...,265,0.438271,-0.346311,0.074514,1.153051,NaT,NaN,NaN,NaN,NaN
5830,crsp_88258,2005-02-28,2005-02-28,NaN,NaN,USA,-0.090909,2005,2,20050131,...,265,NaN,NaN,NaN,-1.606884,NaT,NaN,NaN,NaN,NaN
6131,crsp_89139,2005-02-28,2005-02-28,NaN,NaN,USA,0.269888,2005,2,20050131,...,42,NaN,NaN,NaN,-1.355561,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383497,comp_009919_01,2025-02-28,2025-02-28,9919.0,01,USA,-0.161589,2025,2,20250131,...,505,-0.960044,-0.637429,0.172076,-1.030098,NaT,NaN,NaN,NaN,NaN
1386690,comp_009919_01,2025-03-31,2025-03-31,9919.0,01,USA,-0.192615,2025,3,20250228,...,506,-0.937657,-0.629447,0.194402,-1.015891,NaT,NaN,NaN,NaN,NaN
1389872,comp_009919_01,2025-04-30,2025-04-30,9919.0,01,USA,0.045735,2025,4,20250331,...,507,-0.947467,-0.639770,0.193889,-1.014075,NaT,NaN,NaN,NaN,NaN
1393041,comp_009919_01,2025-05-31,2025-05-31,9919.0,01,USA,0.246311,2025,5,20250430,...,508,-1.181943,-0.537743,-0.500379,-0.944170,NaT,NaN,NaN,NaN,NaN


In [6]:
# Ensure 'date' in raw and 'datadate' in mappings3 are in datetime format
raw['date'] = pd.to_datetime(raw['date'])
mappings3['datadate'] = pd.to_datetime(mappings3['datadate'])

# Find non-overlapping dates in raw
non_overlapping_dates = set(raw['date']) - set(mappings3['datadate'])

# Print the results
if non_overlapping_dates:
    print(f"Number of non-overlapping dates in raw: {len(non_overlapping_dates)}")
    print("Non-overlapping dates:")
    print(non_overlapping_dates)
else:
    print("All dates in raw have a match in mappings3.")

All dates in raw have a match in mappings3.


In [9]:
# Function to check specific rows in mappings3 based on given values for gvkey, date, and iid
def check_specific_rows(mappings3, gvkey, date, iid):
    # Ensure 'datadate' is in datetime format
    mappings3['datadate'] = pd.to_datetime(mappings3['datadate'])

    # Filter rows based on the given values
    specific_rows = mappings3[
        (mappings3['gvkey'] == gvkey) &
        (mappings3['datadate'] == pd.to_datetime(date)) &
        (mappings3['iid'] == iid)
    ]

    # Check if any rows match
    if not specific_rows.empty:
        print(f"Found {len(specific_rows)} matching rows for gvkey={gvkey}, date={date}, iid={iid}:")
        display(specific_rows)  # Use display() in a notebook to inspect the rows
    else:
        print(f"No matching rows found for gvkey={gvkey}, date={date}, iid={iid}.")

# Example usage:
# Replace the values below with the ones you want to check
check_specific_rows(mappings3, gvkey=144498, date='2005-02-28', iid='01C')

No matching rows found for gvkey=144498, date=2005-02-28, iid=01C.
